In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 40.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f25219280226441f00b814222cdfb987bb01298dd403f5900b30b2b28d87350a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this 

In [118]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive as dr
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [62]:
id = '1lm1j7FHp0vrlbnBUmvDsN1LPVYzS7ZTd'
id2 = '1tpUWO3s6NMkcMTAYP_ur2v3Zcwz9yIeI'
downloaded = drive.CreateFile({'id' : id})
downloaded2 = drive.CreateFile({'id' : id2})
downloaded.GetContentFile('train_labels.csv')
downloaded2.GetContentFile('train_values.csv')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica
                                                     # en los outputs

import warnings
warnings.filterwarnings('ignore')

In [77]:
# Lectura acorde a los tipos
train_labels = pd.read_csv("train_labels.csv",\
                           dtype={"damage_grade":'category'})
train_labels.head(1)

,building_id,damage_grade
0,802906,3


In [78]:
train_labels.shape

(260601, 2)

In [80]:
train_labels.dtypes

building_id        int64
damage_grade    category
dtype: object

In [106]:
# Verifico que no haya daños fuera del rango 1 - 3
train_labels['damage_grade'].value_counts()

2    148259
3     87218
1     25124
Name: damage_grade, dtype: int64

In [84]:
# Lectura acorde a los tipos
train_values = pd.read_csv("train_values.csv",\
                           dtype={"land_surface_condition":'category', "foundation_type":'category',\
                                  "roof_type":'category', "ground_floor_type":'category',\
                                  "other_floor_type":'category', "position":'category',\
                                  "plan_configuration":'category',\
                                  "has_superstructure_adobe_mud":'bool',\
                                  "has_superstructure_mud_mortar_stone":'bool',\
                                  "has_superstructure_stone_flag":'bool',\
                                  "has_superstructure_cement_mortar_stone":'bool',\
                                  "has_superstructure_mud_mortar_brick":'bool',\
                                  "has_superstructure_cement_mortar_brick":'bool',\
                                  "has_superstructure_timber":'bool',\
                                  "has_superstructure_bamboo":'bool',\
                                  "has_superstructure_rc_non_engineered":'bool',\
                                  "has_superstructure_rc_engineered":'bool',\
                                  "has_superstructure_other":'bool',\
                                  "legal_ownership_status":'category',\
                                  "has_secondary_use":'bool',\
                                  "has_secondary_use_agriculture":'bool',\
                                  "has_secondary_use_hotel":'bool',\
                                  "has_secondary_use_rental":'bool',\
                                  "has_secondary_use_institution":'bool',\
                                  "has_secondary_use_school":'bool',\
                                  "has_secondary_use_industry":'bool',\
                                  "has_secondary_use_health_post":'bool',\
                                  "has_secondary_use_gov_office":'bool',\
                                  "has_secondary_use_use_police":'bool',\
                                  "has_secondary_use_other":'bool'})
train_values.head(1)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,True,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False


In [85]:
train_values.shape

(260601, 39)

In [87]:
train_values.dtypes

building_id                                  int64
geo_level_1_id                               int64
geo_level_2_id                               int64
geo_level_3_id                               int64
count_floors_pre_eq                          int64
age                                          int64
area_percentage                              int64
height_percentage                            int64
land_surface_condition                    category
foundation_type                           category
roof_type                                 category
ground_floor_type                         category
other_floor_type                          category
position                                  category
plan_configuration                        category
has_superstructure_adobe_mud                  bool
has_superstructure_mud_mortar_stone           bool
has_superstructure_stone_flag                 bool
has_superstructure_cement_mortar_stone        bool
has_superstructure_mud_mortar_b

In [86]:
train_values['building_id'].equals(train_labels['building_id'])

True

In [108]:
# Verifico igualdad entre columnas de dfs
train_values['building_id'].equals(train_labels['building_id'])

True

In [89]:
# Merge de ambos dfs
train_values_damage = train_values.merge(train_labels, how='inner', on='building_id')
train_values_damage

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,True,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,False,True,False,False,False,False,True,True,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,True,False,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,n,f,j,s,q,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2
260597,669485,17,715,2060,2,0,6,5,t,r,n,f,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3
260598,602512,17,51,8163,3,55,6,7,t,r,q,f,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,3
260599,151409,26,39,1851,2,10,14,6,t,r,x,v,s,j,d,False,False,False,False,False,True,False,False,False,False,False,v,1,False,False,False,False,False,False,False,False,False,False,False,2


In [90]:
# Verifico la conservación de los tipos
train_values_damage.dtypes

building_id                                  int64
geo_level_1_id                               int64
geo_level_2_id                               int64
geo_level_3_id                               int64
count_floors_pre_eq                          int64
age                                          int64
area_percentage                              int64
height_percentage                            int64
land_surface_condition                    category
foundation_type                           category
roof_type                                 category
ground_floor_type                         category
other_floor_type                          category
position                                  category
plan_configuration                        category
has_superstructure_adobe_mud                  bool
has_superstructure_mud_mortar_stone           bool
has_superstructure_stone_flag                 bool
has_superstructure_cement_mortar_stone        bool
has_superstructure_mud_mortar_b

In [100]:
# Verifico que valores porcentuales no excedan 100
area_incorrecta = train_values_damage[train_values_damage['area_percentage'] > 100]['area_percentage'].count()
altura_incorrecta = train_values_damage[train_values_damage['height_percentage'] > 100]['area_percentage'].count()
(altura_incorrecta == 0 & area_incorrecta == 0)

True

In [102]:
# Verifico veracidad de datos respecto al recuento de familias
train_values_damage.nlargest(10, 'count_families')

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
68031,569833,26,39,9897,2,0,42,7,t,i,x,v,s,s,u,False,False,False,False,False,False,False,False,False,True,False,v,9,True,False,True,False,False,False,False,False,False,False,False,1
87761,147798,7,157,1763,4,20,57,12,o,i,x,v,s,j,d,False,False,False,False,False,False,False,False,True,False,False,v,9,True,False,True,False,False,False,False,False,False,False,False,2
220496,1031380,20,158,9229,3,10,17,9,t,i,x,v,s,j,d,False,False,False,False,False,True,False,False,True,False,False,v,9,True,False,False,True,False,False,False,False,False,False,False,2
257343,331776,10,1397,3396,1,5,11,3,t,r,n,f,j,s,d,False,False,False,False,False,True,False,False,False,False,False,v,9,False,False,False,False,False,False,False,False,False,False,False,2
171692,930289,26,39,1324,2,25,44,6,t,u,x,v,s,j,d,False,False,False,False,False,False,False,False,False,True,False,v,8,True,False,True,False,False,False,False,False,False,False,False,1
235262,485597,7,157,7679,3,15,20,9,t,i,x,v,s,s,d,False,False,False,False,False,True,False,False,True,False,False,v,8,False,False,False,False,False,False,False,False,False,False,False,2
54909,718208,26,39,2005,2,15,23,7,t,r,x,v,s,s,d,False,False,False,False,False,True,False,False,False,False,False,v,7,True,False,False,True,False,False,False,False,False,False,False,1
97087,106261,3,1387,5122,4,90,9,9,t,r,q,x,q,j,d,True,False,False,False,True,False,False,False,False,False,False,v,7,False,False,False,False,False,False,False,False,False,False,False,2
114389,299955,17,1393,4475,4,40,28,11,t,r,n,f,q,s,d,False,True,False,False,False,False,False,False,False,False,False,v,7,False,False,False,False,False,False,False,False,False,False,False,3
125628,266702,4,1334,9215,2,10,14,8,t,i,n,v,s,j,d,False,False,False,False,False,False,False,False,True,False,False,v,7,True,False,True,False,False,False,False,False,False,False,False,1


In [125]:
train_values_damage.to_csv("train_values_damage.csv")
uploaded = drive.CreateFile({'title': 'train_values_damage.csv'})
uploaded.SetContentFile("train_values_damage.csv")
uploaded.Upload()